In [1]:
import numpy as np
import torch
import torch.nn as nn
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from torch.utils.data import Dataset


class Node:
    def __init__(self, value, func='variable', parents=[], *args):
        self.value = value
        self.func = func
        self.parents = parents
        self.args = args

    def __str__(self):
        return f'({self.value}, {self.func}, {self.args}, {self.parents})'

    def __repr__(self):
        return f'({self.value}, {self.func}, {self.args}, {self.parents})'

In [2]:
def add(a: Node, b: Node):
    # unbox
    va = a.value
    vb = b.value

    # primitive
    vc = np.add(va, vb)

    # box
    c = Node(vc, 'add', [a, b])
    c.args = (va, vb)
    return c

In [3]:
def negative(a: Node):
    # unbox
    va = a.value

    # primitive
    vb = np.negative(va)

    # box
    b = Node(vb, 'negative', [a])
    b.args = (va,)
    return b

In [4]:
def exp(a: Node):
    # unbox
    va = a.value

    # primitive
    vb = np.exp(va)

    # box
    b = Node(vb, 'exp', [a])
    b.args = (va,)
    return b

In [5]:
def reciprocal(a: Node):
    # unbox
    va = a.value

    # primitive
    vb = np.reciprocal(va)

    # box
    b = Node(vb, 'reciprocal', [a])
    b.args = (va,)
    return b

In [6]:
def matmul(a: Node, b: Node):
    va = a.value
    vb = b.value

    vc = va @ vb

    c = Node(vc, 'matmul', [a, b])
    c.args = (va, vb)
    return c

In [7]:
def relu(a: Node):
    va = a.value

    vc = va * (va > 0)

    c = Node(vc, 'relu', [a])
    c.args = (va,)
    return c


a = [1.0, -1.0]
r_a = relu(Node(np.array(a))).value
t_r_a = torch.relu(torch.tensor(a))
np.testing.assert_array_equal(r_a, t_r_a)

In [8]:

def softmax(a: Node):
    va = a.value

    # this would be overflow!
    # exp_va = np.exp(va)
    # vc = exp_va / np.sum(exp_va)

    shiftz = va - np.max(va)
    exps = np.exp(shiftz)
    vc = exps / np.sum(exps)

    c = Node(vc, 'softmax', [a])
    c.args = (va,)
    return c


x = [1., 2, 3]
y = softmax(Node(np.array(x, dtype=np.float32)))
t_x = torch.tensor(x, dtype=torch.float32)
t_y = torch.softmax(t_x, dim=-1)
# todo: why not equal? dtype is the cause
np.testing.assert_allclose(y.value, t_y.cpu().detach().numpy())
np.testing.assert_array_equal(y.value, t_y.cpu().detach().numpy())

In [9]:
def cross_entropy(a: Node, b):
    va = a.value

    shiftz = va - np.max(va)
    exps = np.exp(shiftz)
    vc = -1 * np.log(exps[b] / np.sum(exps))

    c = Node(vc, 'cross_entropy', [a])
    c.args = (va, b)
    return c


va = [1., 2., 3.]
vb = 1
a = Node(np.array(va))
c = cross_entropy(a, vb)

loss = nn.CrossEntropyLoss()
t_a = torch.tensor(va, dtype=torch.float32)
t_c = loss(t_a, torch.tensor(vb))

# todo: why not equal?
np.testing.assert_allclose(c.value, t_c.cpu().detach().numpy(), rtol=1e-7)
np.testing.assert_array_equal(c.value, t_c.cpu().detach().numpy())

AssertionError: 
Arrays are not equal

Mismatched elements: 1 / 1 (100%)
Max absolute difference: 7.79850298e-08
Max relative difference: 5.54026028e-08
 x: array(1.407606)
 y: array(1.407606, dtype=float32)

In [10]:
def logistic(i):
    return reciprocal(add(Node(1), exp(negative(i))))

In [11]:

def backward_pass(g, end_node, vjps):
    tmp_node = Node(end_node.value, parents=[end_node])
    q = []
    gs = {tmp_node: (g,)}
    q.append(tmp_node)
    while len(q) > 0:
        cur_node = q.pop(0)
        cur_gs = gs[cur_node]
        for node, cur_g in zip(cur_node.parents, cur_gs):
            q.append(node)
            vjp = vjps[node.func]
            grads = vjp(cur_g, node.value, *node.args)
            if node not in gs:
                gs[node] = grads
            else:
                gs[node] += grads
    return gs


In [12]:
def add_vjp(g, ans, a, b):
    return g, g

In [13]:
def exp_vjp(g, ans, a):
    return (ans * g,)

In [14]:
def negative_vjp(g, ans, a):
    return (-1 * g,)

In [15]:
def reciprocal_vjp(g, ans, a):
    return (np.divide(-1, a * a) * g,)

In [16]:
def variable_vjp(g, ans):
    return (g,)

In [17]:
vjps = {
    "add": add_vjp,
    "negative": negative_vjp,
    "exp": exp_vjp,
    "reciprocal": reciprocal_vjp,
    "variable": variable_vjp,
}

In [18]:
def relu_vjp(g, ans, a):
    return (ans * g,)


vjps["relu"] = relu_vjp
a = [1., -1.]
g = [1., 1.]
x = Node(np.array(a))
y = relu(x)
t_x = torch.tensor(a, requires_grad=True)
t_y = torch.relu(t_x)

gs = backward_pass(np.array(g), y, vjps)
t_y.backward(torch.tensor([1., 1.]))

np.testing.assert_array_equal(gs[x][0], t_x.grad)


In [19]:
# why? https://deepnotes.io/softmax-crossentropy
def softmax_vjp(g, ans, a):
    dim = len(ans)
    t1 = ans.reshape(dim, 1)
    t2 = ans.reshape(1, dim)
    t3 = t1 @ t2
    t4 = np.zeros((dim, dim))
    for i in range(dim):
        t4[i, i] = ans[i]
    d = t4 - t3

    return (d @ g,)


vjps['softmax'] = softmax_vjp
# x = [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]
# g = np.ones(10, dtype=np.float32)
x = [1, 1, 1]
g = [1., 1, 2]
n_x = Node(np.array(x, dtype=np.float32))
y = softmax(n_x)
t_x = torch.tensor(x, requires_grad=True, dtype=torch.float32)
t_y = torch.softmax(t_x, dim=-1)

t_y.backward(torch.tensor(g))
gs = backward_pass(np.array(g, dtype=np.float32), y, vjps)

np.testing.assert_allclose(gs[n_x][0], t_x.grad, rtol=1e-7)

np.testing.assert_array_equal(gs[n_x][0], t_x.grad)
#assert np.abs(np.sum(gs[n_x][0] - t_x.grad.cpu().detach().numpy())) < 0.0001

AssertionError: 
Arrays are not equal

Mismatched elements: 2 / 3 (66.7%)
Max absolute difference: 7.4505806e-09
Max relative difference: 6.70552159e-08
 x: array([-0.111111, -0.111111,  0.222222])
 y: array([-0.111111, -0.111111,  0.222222], dtype=float32)

In [20]:
def cross_entropy_vjp(g, ans, a, b):
    tmp_a = Node(a)
    ps = softmax(tmp_a).value
    ps[b] = ps[b] - 1

    return (ps,)


vjps['cross_entropy'] = cross_entropy_vjp
va = [1., 2., 3.]
vb = 1
vg = 1.
a = Node(np.array(va))
c = cross_entropy(a, vb)

loss = nn.CrossEntropyLoss()
t_a = torch.tensor(va, requires_grad=True)
t_c = loss(t_a, torch.tensor(vb))

gs = backward_pass(np.array(vg), c, vjps)
t_c.backward(torch.tensor(vg))

np.testing.assert_allclose(gs[a][0], t_a.grad, rtol=1e-7)

np.testing.assert_array_equal(gs[a][0], t_a.grad)

AssertionError: 
Arrays are not equal

Mismatched elements: 3 / 3 (100%)
Max absolute difference: 3.4556622e-08
Max relative difference: 8.42898928e-08
 x: array([ 0.090031, -0.755272,  0.665241])
 y: array([ 0.090031, -0.755271,  0.665241], dtype=float32)

In [21]:
def matmul_vjp(g, ans, a, b):
    # a is a matrix
    # b is a vector
    b_d = np.matmul(np.transpose(a), g)
    r, c = a.shape
    # y=Wx
    # dy/dW = [x;x;x] element wise multi g
    a_d = np.multiply(np.tile(b, (r, 1)), g.reshape(r, 1))

    return (a_d, b_d)


vjps["matmul"] = matmul_vjp

In [22]:
import torch

tz = torch.tensor([1.5, 1.5], requires_grad=True)
# reciprocal(add(Node(1), exp(negative(i))))
y = torch.reciprocal(torch.add(1., torch.exp(torch.negative(tz))))
z = Node(np.array([1.5, 1.5]))
logsit = logistic(z)

y.backward(torch.tensor([1., 1.]))
gs = backward_pass((1., 1.), logsit, vjps)

assert np.sum(tz.grad.cpu().detach().numpy() - gs[z]) < 0.00001

In [23]:
t_x = torch.tensor([1.5, 1.6, 1.7], requires_grad=True)
t_w = torch.tensor([
    [1., 2, 3],
    [4, 5, 6]
], requires_grad=True)
t_b = torch.tensor([0.3, 0.4], requires_grad=True)
t_y = torch.matmul(t_w, t_x) + t_b
t_y.backward(torch.tensor([1., 2.]))

x = Node(np.array([1.5, 1.6, 1.7]))
w = Node(np.array([
    [1., 2, 3],
    [4, 5, 6]
]))
b = Node(np.array([0.3, 0.4]))
y = add(matmul(w, x), b)
gs = backward_pass(np.array([1., 2.]), y, vjps)

assert np.sum(t_x.grad.cpu().detach().numpy() - gs[x]) < 0.00001

In [24]:
t_x = torch.tensor([1.5, 1.6, -1700], requires_grad=True)
t_w = torch.tensor([
    [1., 2, 3],
    [4, 5, 6]
], requires_grad=True)
t_b = torch.tensor([0.3, 0.4], requires_grad=True)
t_y = torch.relu(torch.matmul(t_w, t_x) + t_b)
t_y.backward(torch.tensor([1., 2.]))

x = Node(np.array([1.5, 1.6, -1700]))
w = Node(np.array([
    [1., 2, 3],
    [4, 5, 6]
]))
b = Node(np.array([0.3, 0.4]))
y = relu(add(matmul(w, x), b))
gs = backward_pass(np.array([1., 2.]), y, vjps)

np.testing.assert_array_equal(t_x.grad, gs[x][0])

1. single -> batch
2. python -> cuda
3. dy/dW and grad of softmax

In [48]:
# def mlp(input: Node, hidden_dim=11):
#     v_input = input.value
#     # not support batch yet
#     dim = len(v_input)
#
#     v_w1 = np.ones((hidden_dim, dim))
#     w1 = Node(v_w1)
#     v_b1 = np.ones(hidden_dim)
#     b1 = Node(v_b1)
#     h1 = relu(add(matmul(w1, input), b1))
#     #print('h1: ',h1.value)
#
#     v_w2 = np.ones((10, hidden_dim))
#     w2 = Node(v_w2)
#     #print('w2:', w2)
#     v_b2 = np.ones(10)
#     b2 = Node(v_b2)
#     output = softmax(add(matmul(w2, h1), b2))
#     #print('output', output.value)
#     return output, w1, b1, h1, w2, b2
#
#
# def torch_mlp(input, hidden_dim=10):
#     dim = len(input)
#     #print(input)
#     w1 = torch.autograd.Variable(torch.ones((hidden_dim, dim), dtype=torch.float), requires_grad=True)
#     b1 = torch.autograd.Variable(torch.ones(hidden_dim, dtype=torch.float), requires_grad=True)
#     h1 = torch.relu(torch.matmul(w1, input) + b1)
#     #print('torch h1:',h1)
#
#     w2 = torch.autograd.Variable(torch.ones((10, hidden_dim), dtype=torch.float), requires_grad=True)
#     #print('torch w2:', w2)
#     b2 = torch.autograd.Variable(torch.ones(10, dtype=torch.float), requires_grad=True)
#     output = torch.softmax(torch.matmul(w2, h1) + b2, dim=-1)
#     #print('torch output:', output)
#     return output, w1, b1, h1, w2, b2

class Mlp(object):
    def __init__(self, input_dim, hidden_dim):
        v_w1 = np.random.randn(hidden_dim, input_dim)
        self.w1 = Node(v_w1)
        v_b1 = np.random.randn(hidden_dim)
        self.b1 = Node(v_b1)
        v_w2 = np.random.randn(10, hidden_dim)
        self.w2 = Node(v_w2)
        #print('w2:', w2)
        v_b2 = np.random.randn(10)
        self.b2 = Node(v_b2)

    def __call__(self, input):
        h1 = relu(add(matmul(self.w1, input), self.b1))
        output = softmax(add(matmul(self.w2, h1), self.b2))
        return output

    def update(self, gs):
        learning_rate = 0.001
        self.b2.value -= learning_rate * gs[self.b2][0]
        self.w2.value -= learning_rate * gs[self.w2][0]
        self.b1.value -= learning_rate * gs[self.b1][0]
        self.w1.value -= learning_rate * gs[self.w1][0]


# initialize with all ones won't work!
class TorchMlp(object):
    def __init__(self, input_dim, hidden_dim):
        self.w1 = torch.autograd.Variable(torch.randn((hidden_dim, input_dim), dtype=torch.float), requires_grad=True)
        self.b1 = torch.autograd.Variable(torch.randn(hidden_dim, dtype=torch.float), requires_grad=True)
        self.w2 = torch.autograd.Variable(torch.randn((10, hidden_dim), dtype=torch.float), requires_grad=True)
        self.b2 = torch.autograd.Variable(torch.randn(10, dtype=torch.float), requires_grad=True)
        self.cost_func = nn.CrossEntropyLoss()

    def __call__(self, x):
        h1 = torch.relu(torch.matmul(self.w1, x) + self.b1)
        output = torch.softmax(torch.matmul(self.w2, h1) + self.b2, dim=-1)
        return output

    def update(self):
        learning_rate = 0.00001
        self.b2.data -= learning_rate * self.b2.grad
        self.w2.data -= learning_rate * self.w2.grad
        self.b1.data -= learning_rate * self.b1.grad
        self.w1.data -= learning_rate * self.w1.grad

In [39]:
input_len = 18
hidden_dim = 11
input_value = np.arange(0, input_len, dtype=np.float32)
input = Node(input_value)
mlp = Mlp(input_len, hidden_dim)
y = mlp(input)
#y, w1, b1, h1, w2, b2 = mlp(input)

t_input = torch.tensor(input_value, requires_grad=True)
torch_mlp = TorchMlp(input_len, hidden_dim)
t_y = torch_mlp(t_input)
#t_y, t_w1, t_b1, t_h1, t_w2, t_b2 = torch_mlp(t_input)

# assert gradient
initial_g = np.ones(10, dtype=np.float32)
gs = backward_pass(initial_g, y, vjps)
t_y.backward(torch.tensor(initial_g))

# why  so big difference between the gradients
np.testing.assert_allclose(t_y.cpu().detach().numpy(), y.value, rtol=1e-7)
np.testing.assert_allclose(t_input.grad, gs[input][0], rtol=1e-5)

np.testing.assert_array_equal(t_y.cpu().detach().numpy(), y.value)
np.testing.assert_array_equal(t_input.grad, gs[input][0])

AssertionError: 
Not equal to tolerance rtol=1e-07, atol=0

Mismatched elements: 10 / 10 (100%)
Max absolute difference: 1.
Max relative difference: 9.70028294e+133
 x: array([0.000000e+00, 0.000000e+00, 3.499920e-28, 1.000000e+00,
       6.492693e-31, 0.000000e+00, 1.602200e-31, 1.404348e-29,
       2.123127e-18, 2.668722e-19], dtype=float32)
 y: array([1.000000e+000, 6.682237e-142, 9.438314e-114, 1.445316e-110,
       6.693303e-165, 8.381899e-063, 2.139587e-137, 3.800375e-124,
       4.257392e-110, 4.782700e-097])

In [27]:
import numpy as np
from urllib import request
import gzip
import pickle

filename = [
    ["training_images", "train-images-idx3-ubyte.gz"],
    ["test_images", "t10k-images-idx3-ubyte.gz"],
    ["training_labels", "train-labels-idx1-ubyte.gz"],
    ["test_labels", "t10k-labels-idx1-ubyte.gz"]
]


def download_mnist():
    base_url = "http://yann.lecun.com/exdb/mnist/"
    for name in filename:
        print("Downloading " + name[1] + "...")
        request.urlretrieve(base_url + name[1], name[1])
    print("Download complete.")


def save_mnist():
    mnist = {}
    for name in filename[:2]:
        with gzip.open(name[1], 'rb') as f:
            mnist[name[0]] = np.frombuffer(f.read(), np.uint8, offset=16).reshape(-1, 28 * 28)
    for name in filename[-2:]:
        with gzip.open(name[1], 'rb') as f:
            mnist[name[0]] = np.frombuffer(f.read(), np.uint8, offset=8)
    with open("mnist.pkl", 'wb') as f:
        pickle.dump(mnist, f)
    print("Save complete.")


def init():
    download_mnist()
    save_mnist()


def load():
    with open("mnist.pkl", 'rb') as f:
        mnist = pickle.load(f)
    return mnist["training_images"], mnist["training_labels"], mnist["test_images"], mnist["test_labels"]


In [28]:
init()
x_train, y_train, x_test, y_test = load()

Download complete.
Save complete.


Train torch MLP

In [32]:
def normalize(x):
    min = np.min(x)
    max = np.max(x)
    return (x - min) / (max - min)


def cal_acc(preds, trues):
    return np.sum(preds == trues) / len(preds)


epoch_num = 1
learning_rate = 1
torch_mlp = TorchMlp(len(x_train[0]), 128)
cost_func = nn.CrossEntropyLoss()
losses = []
normalized_x_train = normalize(x_train)
normalized_x_test=normalize(x_test)
for i in range(epoch_num):
    print('epoch: ', i)
    for j in range(len(y_train)):
        ind = np.random.choice(len(y_train), 1)[0]
        t_x = torch.tensor(normalized_x_train[ind], dtype=torch.float)
        t_y = torch_mlp(t_x)
        #print(t_y)
        loss = cost_func(t_y, torch.tensor(y_train[ind]))
        #print(loss)
        # seems wrong? loss always reduce? maybe less than 0
        loss.backward()
        torch_mlp.update()
        losses.append(loss.cpu().detach().numpy())

        if j % 1000 == 999:
            print('loss', loss)
            losses.append(loss.cpu().detach().numpy())
            preds = []
            for x in normalized_x_test:
                output = torch_mlp(torch.tensor(x, dtype=torch.float))
                pred = torch.argmax(output).cpu().detach().numpy()
                preds.append(pred)

            acc = cal_acc(preds, y_test)
            print('test acc: ', acc)
            # print('train acc: ', train_acc)
            # test_preds = torch_mlp(torch.tensor(x_test, dtype=torch.float))
            # test_acc = accuracy_score(torch.argmax(test_preds, dim=1).cpu().detach().numpy(), y_test)
            # print('test acc: ', test_acc)



epoch:  0
loss tensor(2.4612, grad_fn=<NllLossBackward0>)
test acc:  0.1384
loss tensor(2.4612, grad_fn=<NllLossBackward0>)
test acc:  0.1884
loss tensor(2.4612, grad_fn=<NllLossBackward0>)
test acc:  0.2418
loss tensor(2.4612, grad_fn=<NllLossBackward0>)
test acc:  0.2625
loss tensor(2.4610, grad_fn=<NllLossBackward0>)
test acc:  0.284
loss tensor(2.4612, grad_fn=<NllLossBackward0>)
test acc:  0.299
loss tensor(2.4612, grad_fn=<NllLossBackward0>)
test acc:  0.3071
loss tensor(2.4612, grad_fn=<NllLossBackward0>)
test acc:  0.3135
loss tensor(1.4612, grad_fn=<NllLossBackward0>)
test acc:  0.3231
loss tensor(2.4612, grad_fn=<NllLossBackward0>)
test acc:  0.3342
loss tensor(2.4564, grad_fn=<NllLossBackward0>)
test acc:  0.3488
loss tensor(2.4611, grad_fn=<NllLossBackward0>)
test acc:  0.3455
loss tensor(2.4612, grad_fn=<NllLossBackward0>)
test acc:  0.3412
loss tensor(1.4612, grad_fn=<NllLossBackward0>)
test acc:  0.3376
loss tensor(2.4612, grad_fn=<NllLossBackward0>)
test acc:  0.3364
lo

In [51]:
epoch_num = 1
learning_rate = 1
mlp = Mlp(len(x_train[0]), 128)
cost_func = cross_entropy
losses = []

for i in range(epoch_num):
    print('epoch: ', i)
    for j in range(len(y_train)):
        ind = np.random.choice(len(y_train), 1)[0]
        x=Node(normalized_x_train[ind])
        y = mlp(x)
        #print(t_y)
        loss = cost_func(y, y_train[ind])
        #print(loss)
        # seems wrong? loss always reduce? maybe less than 0
        gs = backward_pass(np.array(1), loss, vjps)
        mlp.update(gs)

        if j % 1000 == 999:
            print('loss', loss.value)
            losses.append(loss.value)
            preds = []
            for x in normalized_x_test:
                output = mlp(Node(x))
                pred = np.argmax(output.value)
                preds.append(pred)

            acc = cal_acc(preds, y_test)
            print('test acc: ', acc)

epoch:  0
loss 2.4611199590323865
test acc:  0.104
loss 2.4611501149060215
test acc:  0.1144
loss 2.4611501717202247
test acc:  0.1476
loss 2.461147882711231
test acc:  0.1575
loss 2.3963732048291693
test acc:  0.1684
loss 2.461150171734475
test acc:  0.1853
loss 1.4611501717344748
test acc:  0.1972
loss 2.461149134214046
test acc:  0.2126
loss 2.461150171734475
test acc:  0.2266
loss 2.4396076116421774
test acc:  0.2441
loss 1.461262969706156
test acc:  0.2662
loss 1.4611501718659807
test acc:  0.2754
loss 2.4611472095233426
test acc:  0.2844
loss 1.4611501717344748
test acc:  0.2976
loss 1.4611501717344748
test acc:  0.3083
loss 1.4611501717344964
test acc:  0.3241
loss 1.4611501717344797
test acc:  0.3367
loss 2.4611501666849778
test acc:  0.3454
loss 2.461150171734414
test acc:  0.3606
loss 1.4611501717344748
test acc:  0.3624
loss 2.461150166678513
test acc:  0.3782
loss 1.4611501718673654
test acc:  0.3883
loss 2.4172144703676848
test acc:  0.39
loss 1.4611501717344748
test acc: 

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(losses)
np.min(losses)

In [ ]:
torch_mlp.b2.grad

In [ ]:
torch_mlp.w2.grad

In [ ]:
class TorchRealMlp(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.linear1 = nn.Linear(input_dim, 128)
        self.relu1 = nn.ReLU()
        self.linear2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.linear1(x)
        x = self.relu1(x)
        x = self.linear2(x)
        return torch.softmax(x, dim=-1)


class MnistDataset(Dataset):
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]




In [ ]:
import torch.optim as optim

torch_real_mlp = TorchRealMlp(len(x_train[0]))

# for param in torch_real_mlp.parameters():
#     param.data = nn.parameter.Parameter(torch.ones_like(param))
for param in torch_real_mlp.parameters():
    param.data = nn.parameter.Parameter(torch.randn_like(param))

print(torch_real_mlp)
cost_func = nn.CrossEntropyLoss()

optimizer = optim.SGD(torch_real_mlp.parameters(), lr=0.01)
losses = []
learning_rate = 1
epoch_num = 6
batch_size = 1
dataset = MnistDataset(x_train, y_train)
train_loader = torch.utils.data.DataLoader(dataset=dataset,
                                           batch_size=batch_size,
                                           shuffle=True)
normalized_x_train = normalize(x_train)  # must be normalized, or else not trainable
# for batch_idx, (data, target) in enumerate(train_loader):
#     print(data)
#     print(target)
#     # Zero gradient buffers
#     optimizer.zero_grad()
#
#     # Pass data through the network
#     output = torch_real_mlp(data)
#
#     # Calculate loss
#     loss = cost_func(output, target)
#
#     # Backpropagate
#     loss.backward()
#
#     # Update weights
#     optimizer.step()
#
#     if j % 1000 == 999:
#         torch_real_mlp.eval()
#         print('loss', loss)
#         losses.append(loss.cpu().detach().numpy())
#         train_acc = accuracy_score(torch.argmax(t_y, dim=1).cpu().detach().numpy(), y_train[ind])
#         print('train acc: ', train_acc)
#         test_preds = torch_real_mlp(torch.tensor(x_test, dtype=torch.float))
#         test_acc = accuracy_score(torch.argmax(test_preds, dim=1).cpu().detach().numpy(), y_test)
#         print('test acc: ', test_acc)


for i in range(epoch_num):
    for j in range(len(y_train) // batch_size):
        torch_real_mlp.train()
        optimizer.zero_grad()
        ind = np.random.choice(len(y_train), batch_size)
        t_x = torch.tensor(normalized_x_train[ind], dtype=torch.float)
        t_y = torch_real_mlp(t_x)
        labels = torch.tensor(y_train[ind])

        loss = cost_func(t_y, labels)
        #
        # print('================begin=================', j)
        # plt.imshow(x_train[ind].reshape(28,28))
        # print(labels)
        # print('label: ', y_train[ind])
        # print('t_y: ', t_y)
        # print('loss: ', loss)
        # print('--------------------------------')
        #seems wrong? loss always reduce? maybe less than 0
        loss.backward()
        optimizer.step()
        # if j % 100 == 99:
        #     print('------------------------------')
        #     t_y = torch_real_mlp(t_x)
        #     print('t_y: ', t_y)
        #     print('loss: ', cost_func(t_y, torch.tensor(y_train[ind])))
        #     print('================end=================')
        if j % 1000 == 999:
            torch_real_mlp.eval()
            print(f'loss{loss}/{i}', loss)
            losses.append(loss.cpu().detach().numpy())
            train_acc = accuracy_score(torch.argmax(t_y, dim=1).cpu().detach().numpy(), y_train[ind])
            print('train acc: ', train_acc)
            test_preds = torch_real_mlp(torch.tensor(x_test, dtype=torch.float))
            test_acc = accuracy_score(torch.argmax(test_preds, dim=1).cpu().detach().numpy(), y_test)
            print('test acc: ', test_acc)

plt.plot(losses)

In [ ]:
from sklearn.metrics import accuracy_score

accuracy_score(torch.argmax(t_y, dim=1).cpu().detach().numpy(), y_train[ind])

In [ ]:
# loss = nn.CrossEntropyLoss()
# label = 4
# y = torch.tensor([1.6835e-02, 2.7039e-04, 1.6209e-01, 2.7039e-04, 2.7039e-04, 2.7039e-04,
#                   2.7039e-04, 1.0372e-03, 8.1841e-01, 2.7039e-04])
# print(loss(y, torch.tensor(label)))
# sy = torch.softmax(y, dim=-1)
# -1 * torch.log(sy[label] / torch.sum(sy))

In [ ]:
plt.plot(losses)